In [1]:
%pathsetup lib/python

"Updated sys path to include repository root and as well as subdirs ['lib/python']"

In [2]:
import numpy as np
import pandas as pd
#import rmm_util_accessor
import itertools
import time
import re
import grid

In [34]:
day = 8
#infile = f'/home/rmohring/code/rmm/advent-rmm-2024/source/day{day:02}/{day}-1-test.dat'
infile = f'/home/rmohring/code/rmm/advent-rmm-2024/source/day{day:02}/{day}-1.dat'
#infile = f'/home/rmohring/code/rmm/advent-rmm-2024/source/day{day:02}/{day}-2.dat'
data = []
with open(infile, 'r') as fh:
    data = fh.readlines()

In [35]:
g = grid.Gridder(data, from_text=True)
g.shape

(50, 50)

In [36]:
g.pp()

.....................U.........w..................
l.................................................
...........o.a................U...w...............
............................................W.....
..........T....................s.............7....
.............................................W....
.........T..............4....n.d.H.........5......
......T.....oj...U.....n...w......H...........z...
.G..x..........................E.....V..H.........
.........a....................d....s.......7w.....
...j....r.............o.............V.......d...W.
.......r..J.Goa.U...............n................z
.........Jj.........M..........Pv.................
...J...........t..3..M..............sLV...........
...................t................n.............
....r...........X...........M........v............
...x....t......I......a.PM...............W........
...........1.Bj....I........vO.h.dL...............
.........6....Rr......B...X........h..5v.L..z.....
......1G...........x.....3B....

In [37]:
ant_types = set(g.grid.ravel())
ant_types.remove(".")

In [38]:
ant_types

{'0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [39]:
q = np.where(g.grid=='0')
list(zip(q[0],q[1]))

[(20, 22), (43, 8), (46, 13), (47, 11)]

In [40]:
def get_antinodes(p1, p2):
    
    dr = p2[0]-p1[0]
    dc = p2[1]-p1[1]
    
    a1 = (p1[0]-dr, p1[1]-dc)
    a2 = (p2[0]+dr, p2[1]+dc)
    return (a1, a2)    

In [41]:
get_antinodes((4,4),(3,3))

((5, 5), (2, 2))

In [42]:
#g = grid.Gridder(shape=g.shape)
#g.setval((3,4),"8")
#g.setval((4,6),"8")

In [43]:
ant_types = set(g.grid.ravel())
ant_types.remove(".")
occupied = set()
anodes = set()
#testval = 1
#for a in list(ant_types)[testval:testval+1]:
viz = grid.Gridder(shape=g.grid.shape)
for a in list(ant_types):    
    #viz = grid.Gridder(shape=g.grid.shape)
    q = np.where(g.grid==a)
    alocs = list(zip(q[0],q[1]))
    viz.setvals(alocs, a)
    occupied.update(alocs)
    print(f"Antenna: {a} ({len(alocs)} inst)")
    this_anodes = set()
    for i in range(len(alocs)):
        for j in range(i+1, len(alocs)):
            print(i,j,alocs[i],alocs[j], get_antinodes(alocs[i],alocs[j]))
            this_anodes.update(get_antinodes(alocs[i],alocs[j]))
            anodes.update(get_antinodes(alocs[i],alocs[j]))

    for x in this_anodes:
        if viz.is_ongrid(x):
            viz.setval(x, "#")
        else:
            print(f"{x} is offgrid")

Antenna: X (4 inst)
0 1 (15, 16) (18, 26) ((12, 6), (21, 36))
0 2 (15, 16) (21, 21) ((9, 11), (27, 26))
0 3 (15, 16) (28, 13) ((2, 19), (41, 10))
1 2 (18, 26) (21, 21) ((15, 31), (24, 16))
1 3 (18, 26) (28, 13) ((8, 39), (38, 0))
2 3 (21, 21) (28, 13) ((14, 29), (35, 5))
Antenna: R (4 inst)
0 1 (18, 14) (23, 23) ((13, 5), (28, 32))
0 2 (18, 14) (29, 22) ((7, 6), (40, 30))
0 3 (18, 14) (30, 19) ((6, 9), (42, 24))
1 2 (23, 23) (29, 22) ((17, 24), (35, 21))
1 3 (23, 23) (30, 19) ((16, 27), (37, 15))
2 3 (29, 22) (30, 19) ((28, 25), (31, 16))
Antenna: o (4 inst)
0 1 (2, 11) (7, 12) ((-3, 10), (12, 13))
0 2 (2, 11) (10, 22) ((-6, 0), (18, 33))
0 3 (2, 11) (11, 13) ((-7, 9), (20, 15))
1 2 (7, 12) (10, 22) ((4, 2), (13, 32))
1 3 (7, 12) (11, 13) ((3, 11), (15, 14))
2 3 (10, 22) (11, 13) ((9, 31), (12, 4))
(-3, 10) is offgrid
(-6, 0) is offgrid
(-7, 9) is offgrid
Antenna: f (4 inst)
0 1 (21, 40) (26, 47) ((16, 33), (31, 54))
0 2 (21, 40) (30, 44) ((12, 36), (39, 48))
0 3 (21, 40) (35, 43) ((7,

In [44]:
viz.pp()

....##....#........#.U.........w..........#.......
l.............#.#.........##...............#......
.##........#.a..#.##..........#...w...............
...........#..#..##........#....#...#......#W.....
..#.......T............#..#.#..s.....#.#.....7....
..........#.#...................#.##.........W....
......#..T..............4#..#n.d.H..#.##...5......
....#.#...##oj#..U.....n...w......H.##........z#..
.G.#x...#.#....###..........#..E.###.V##H.........
.........#.##....#.........#..d#...s....#..7w.#...
..#j....r..#.....#....#.........##..V#......d..#W.
.......r..J.G###U..##...........n................z
#...#.#..Jj#.#.#....#.....#..##P#####.............
...J.#..#......t.#3#.M..........#...sLV#.#....#...
........##...#.###.t..##.....#.##...#..#.#........
#...r...#..#..###......#....M.##.....#..##.......#
...x##..t......I.#....#.PM.#.....#.#..#.#W........
......##...1.Bj...#I..#.#.#.vO.h.dL###..###.......
#.#..#...6....Rr......B#..X......####.5v.L#.z.....
.#....1G.......#...#.....3B#.#.

In [45]:
len(anodes)

609

In [46]:
for x in sorted(anodes):
    if g.is_offgrid(x):
        print(x)

(-43, -5)
(-36, -2)
(-26, -1)
(-24, 28)
(-24, 68)
(-17, 37)
(-15, 23)
(-14, 36)
(-14, 64)
(-13, 12)
(-13, 55)
(-12, 4)
(-11, 26)
(-11, 57)
(-10, 47)
(-9, 18)
(-9, 20)
(-9, 59)
(-9, 69)
(-8, -4)
(-8, 15)
(-7, 9)
(-7, 12)
(-7, 25)
(-7, 33)
(-7, 35)
(-7, 44)
(-7, 52)
(-6, 0)
(-6, 31)
(-6, 38)
(-6, 48)
(-6, 49)
(-5, 17)
(-5, 24)
(-5, 26)
(-5, 29)
(-5, 55)
(-4, 20)
(-4, 26)
(-4, 40)
(-4, 48)
(-4, 53)
(-3, -11)
(-3, -5)
(-3, 4)
(-3, 10)
(-3, 12)
(-3, 36)
(-3, 41)
(-3, 43)
(-2, 10)
(-2, 12)
(-2, 18)
(-2, 22)
(-2, 28)
(-1, 7)
(-1, 12)
(-1, 27)
(-1, 28)
(-1, 47)
(0, 62)
(2, -4)
(3, -10)
(3, -8)
(3, -2)
(3, 55)
(4, -10)
(4, -1)
(4, 54)
(4, 55)
(5, -10)
(8, -4)
(11, -3)
(11, 58)
(11, 61)
(12, -7)
(13, -13)
(13, -7)
(14, -30)
(14, -3)
(14, 57)
(15, -25)
(15, -4)
(15, 51)
(15, 52)
(16, 54)
(17, 51)
(17, 52)
(17, 59)
(18, -3)
(18, -2)
(18, 57)
(19, 50)
(22, 50)
(22, 53)
(23, -6)
(24, -17)
(24, 54)
(25, -1)
(28, 70)
(29, 76)
(30, -9)
(30, 61)
(31, -1)
(31, 54)
(32, -6)
(32, -5)
(34, 56)
(42, -18)
(42

In [47]:
anodes_ongrid = set(x for x in anodes if g.is_ongrid(x))
len(anodes_ongrid)

423

ctr = 0
for a in sorted(anodes_ongrid):
    if a in occupied:
        print(f"{a} {g.val(a)}")
    else:
        print(a)
        ctr += 1

ctr

# Part 2

In [53]:
def get_antinodes_2(p1, p2):   
    dr = p2[0]-p1[0]
    dc = p2[1]-p1[1]

    ans = set([p1, p2])
    
    for n in range(1, 50):
        a1 = (p1[0]-n*dr, p1[1]-n*dc)
        a2 = (p2[0]+n*dr, p2[1]+n*dc)
        ans.add(a1)
        ans.add(a2)
    return ans    

In [54]:
get_antinodes_2((4,4),(3,3))

{(-46, -46),
 (-45, -45),
 (-44, -44),
 (-43, -43),
 (-42, -42),
 (-41, -41),
 (-40, -40),
 (-39, -39),
 (-38, -38),
 (-37, -37),
 (-36, -36),
 (-35, -35),
 (-34, -34),
 (-33, -33),
 (-32, -32),
 (-31, -31),
 (-30, -30),
 (-29, -29),
 (-28, -28),
 (-27, -27),
 (-26, -26),
 (-25, -25),
 (-24, -24),
 (-23, -23),
 (-22, -22),
 (-21, -21),
 (-20, -20),
 (-19, -19),
 (-18, -18),
 (-17, -17),
 (-16, -16),
 (-15, -15),
 (-14, -14),
 (-13, -13),
 (-12, -12),
 (-11, -11),
 (-10, -10),
 (-9, -9),
 (-8, -8),
 (-7, -7),
 (-6, -6),
 (-5, -5),
 (-4, -4),
 (-3, -3),
 (-2, -2),
 (-1, -1),
 (0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (25, 25),
 (26, 26),
 (27, 27),
 (28, 28),
 (29, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (38, 38),
 (39, 39),


In [56]:
ant_types = set(g.grid.ravel())
ant_types.remove(".")
occupied = set()
anodes = set()
#testval = 1
#for a in list(ant_types)[testval:testval+1]:
viz = grid.Gridder(shape=g.grid.shape)
for a in list(ant_types):    
    #viz = grid.Gridder(shape=g.grid.shape)
    q = np.where(g.grid==a)
    alocs = list(zip(q[0],q[1]))
    viz.setvals(alocs, a)
    occupied.update(alocs)
    print(f"Antenna: {a} ({len(alocs)} inst)")
    for i in range(len(alocs)):
        for j in range(i+1, len(alocs)):
            print(i,j,alocs[i],alocs[j], len(get_antinodes_2(alocs[i],alocs[j])))
            anodes.update(get_antinodes_2(alocs[i],alocs[j]))


Antenna: X (4 inst)
0 1 (15, 16) (18, 26) 100
0 2 (15, 16) (21, 21) 100
0 3 (15, 16) (28, 13) 100
1 2 (18, 26) (21, 21) 100
1 3 (18, 26) (28, 13) 100
2 3 (21, 21) (28, 13) 100
Antenna: R (4 inst)
0 1 (18, 14) (23, 23) 100
0 2 (18, 14) (29, 22) 100
0 3 (18, 14) (30, 19) 100
1 2 (23, 23) (29, 22) 100
1 3 (23, 23) (30, 19) 100
2 3 (29, 22) (30, 19) 100
Antenna: o (4 inst)
0 1 (2, 11) (7, 12) 100
0 2 (2, 11) (10, 22) 100
0 3 (2, 11) (11, 13) 100
1 2 (7, 12) (10, 22) 100
1 3 (7, 12) (11, 13) 100
2 3 (10, 22) (11, 13) 100
Antenna: f (4 inst)
0 1 (21, 40) (26, 47) 100
0 2 (21, 40) (30, 44) 100
0 3 (21, 40) (35, 43) 100
1 2 (26, 47) (30, 44) 100
1 3 (26, 47) (35, 43) 100
2 3 (30, 44) (35, 43) 100
Antenna: V (3 inst)
0 1 (8, 37) (10, 36) 100
0 2 (8, 37) (13, 38) 100
1 2 (10, 36) (13, 38) 100
Antenna: a (4 inst)
0 1 (2, 13) (9, 9) 100
0 2 (2, 13) (11, 14) 100
0 3 (2, 13) (16, 22) 100
1 2 (9, 9) (11, 14) 100
1 3 (9, 9) (16, 22) 100
2 3 (11, 14) (16, 22) 100
Antenna: U (4 inst)
0 1 (0, 21) (2, 30)

In [57]:
anodes_ongrid = set(x for x in anodes if g.is_ongrid(x))
len(anodes_ongrid)

1287